Importing datasets into pandas

In [148]:
import matplotlib as plt
import kaggle
import zipfile
import pandas as pd
import numpy as np
import os

In [154]:
#downloading pollution_us dataset if doesnt exist locally
us_pollution_filename = 'pollution_us_2000_2016.csv'
if not(os.path.isfile(us_pollution_filename)):
    kaggle.api.dataset_download_file('sogun3/uspollution', us_pollution_filename)
    zip_ref = zipfile.ZipFile(us_pollution_filename+'.zip', 'r')
    zip_ref.extractall()
    zip_ref.close()
df_us_pollution = pd.read_csv(us_pollution_filename)

In [142]:
#assuming resp dataset already exists locally
resp_data_path = 'IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014_NATIONAL_Y2017M09D26.XLSX'
list_sheet_names = ['Chronic respiratory diseases', 'Chronic obstructive pulmonary ', 'Pneumoconiosis', 'Silicosis',
                   'Asbestosis', 'Coal workers pneumoconiosis', 'Other pneumoconiosis', 'Asthma', 'Interstitial lung disease',
                   'Other chronic respiratory ']
df_resp_disease = pd.read_excel(resp_data_path, skiprows=1, sheet_name=list_sheet_names)


In [144]:
for x in list_sheet_names:
    #dropping mortality rate measured below year 2000
    #dropping any na values
    df_resp_disease[x].drop(['Mortality Rate, 1980*', 'Mortality Rate, 1985*', 'Mortality Rate, 1990*', 'Mortality Rate, 1995*'],
                           inplace=True, errors='ignore', axis=1)
    df_resp_disease[x].dropna(inplace=True)
    #Extracting state out of Location
    df_resp_disease[x]['state'] = df_resp_disease[x]['Location']\
    .apply(lambda x: x.split(',')[1].strip() if len(x.split(',')) > 1 else x.strip())

In [105]:
#df_us_pollution does not have data for all counties, therefore need to do it based on state
#need to aggregate by year in date local field to match with mortality data ([2000, 2005, 2010, 2014])
df_us_pollution['year'] = pd.DatetimeIndex(df_us_pollution['Date Local']).year
series_year_data = df_us_pollution['year']
df_us_pollution['year_bin'] = np.where(np.logical_and(series_year_data>=2000, series_year_data<2005), 2000,
                                      np.where(np.logical_and(series_year_data>=2005, series_year_data<2010), 2005,
                                      np.where(series_year_data>=2010, 2010, None)))

In [139]:
df_resp_disease[list_sheet_names[0]].groupby(['state'])

In [145]:
df_resp_disease

OrderedDict([('Chronic respiratory diseases',
                                        Location     FIPS Mortality Rate, 2000*  \
              1                          Alabama      1.0  65.65 (64.23, 67.09)   
              2          Autauga County, Alabama   1001.0  71.77 (68.27, 75.56)   
              3          Baldwin County, Alabama   1003.0  52.07 (49.85, 54.48)   
              4          Barbour County, Alabama   1005.0  65.19 (61.87, 68.69)   
              5             Bibb County, Alabama   1007.0  74.13 (69.86, 78.72)   
              6           Blount County, Alabama   1009.0  71.95 (68.40, 75.50)   
              7          Bullock County, Alabama   1011.0  50.81 (46.83, 55.06)   
              8           Butler County, Alabama   1013.0  60.60 (57.06, 64.18)   
              9          Calhoun County, Alabama   1015.0  77.82 (74.87, 80.95)   
              10        Chambers County, Alabama   1017.0  68.06 (65.07, 71.54)   
              11        Cherokee County, 